In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.nn.functional as F

In [0]:
model = models.resnet50(pretrained = True)

In [121]:
model

ResNet(
  (conv1): Conv2d (3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d (64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (conv2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (conv3): Conv2d (64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d (64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      )
    )
    (1):

In [0]:
class myResNet(nn.Module):
            def __init__(self):
                super(myResNet, self).__init__()
                self.features = nn.Sequential(
                    # Stops at AvgPool2d
                    *list(model.children())[:-1]
                )
            def forward(self, x):
                x = self.features(x)
                return x
# Alternative
# res50_conv = nn.Sequential(*list(model.children())[:-1])

In [0]:
res = myResNet()

In [124]:
res

myResNet(
  (features): Sequential(
    (0): Conv2d (3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d (64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
        (conv2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
        (conv3): Conv2d (64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
        (relu): ReLU(inplace)
        (downsample): Sequential(
          (0): Conv2d (64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(256, eps=1e-05, momentum=

In [125]:
input  = torch.autograd.Variable(torch.Tensor(1, 3, 512, 512))
out = res.forward(input)
out.size()

torch.Size([1, 2048, 10, 10])

In [126]:
# Modify the orginal models accordingly
model.fc = torch.nn.Linear(204800, 10)
out2 = model.forward(input)
out2.size()

torch.Size([1, 10])

In [127]:
model

ResNet(
  (conv1): Conv2d (3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d (64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (conv2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (conv3): Conv2d (64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d (64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      )
    )
    (1):

In [0]:
# Multiple Learning Rate

optimizer = torch.optim.Adam([
        {'params': model.conv1.parameters()},
        {'params': model.bn1.parameters()},
        {'params': model.relu.parameters()},
        {'params': model.maxpool.parameters()},
        {'params': model.layer1.parameters()},
        {'params': model.layer2.parameters()},
        {'params': model.layer3.parameters()},
        {'params': model.layer4.parameters()},
        {'params': model.avgpool.parameters()},
        {'params': model.fc.parameters(),
         'lr': 0.001}], lr=0.0001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)

VGG 16

In [0]:
# VGG16

model = models.vgg16(pretrained = True)

In [138]:
model

VGG(
  (features): Sequential(
    (0): Conv2d (3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (5): Conv2d (64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d (128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (10): Conv2d (128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (17): Conv2d (256, 512, kernel_size=(3, 3), 

In [0]:
class myVGG(nn.Module):
  def __init__(self):
      super(myVGG, self).__init__()
      self.features = nn.Sequential(
          # Stops at classifier
          *list(model.features.children())
      )
  def forward(self, x):
      x = self.features(x)
      return x

In [0]:
vgg = myVGG()

In [141]:
vgg

myVGG(
  (features): Sequential(
    (0): Conv2d (3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (5): Conv2d (64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d (128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (10): Conv2d (128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (17): Conv2d (256, 512, kernel_size=(3, 3)

In [0]:
input  = torch.autograd.Variable(torch.Tensor(1, 3, 64, 64))
out = vgg.forward(input)

In [143]:
out.size()

torch.Size([1, 512, 2, 2])

In [0]:
# Just to change the last one
# mod = list(model.classifier.children()) 
# mod.pop()
# mod.append(torch.nn.Linear(512*2*2, 10))
# new_classifier = torch.nn.Sequential(*mod)
# model.classifier = new_classifier

# To change the whole classifier
mode = []
mode.append(torch.nn.Linear(512*2*2, 4096))
mode.append(torch.nn.ReLU())
mode.append(torch.nn.Dropout(p=0.5))
mode.append(torch.nn.Linear(4096, 4096))
mode.append(torch.nn.ReLU())
mode.append(torch.nn.Dropout(p=0.5))
mode.append(torch.nn.Linear(4096, 10))
new_classifier = torch.nn.Sequential(*mode)
model.classifier = new_classifier

In [145]:
model

VGG(
  (features): Sequential(
    (0): Conv2d (3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (5): Conv2d (64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d (128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (10): Conv2d (128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (17): Conv2d (256, 512, kernel_size=(3, 3), 

In [146]:
out2 = model.forward(input)
out2.size()

torch.Size([1, 10])

In [0]:
# Multiple learning rates
optimizer = torch.optim.Adam([
        {'params': model.features.parameters()},
        {'params': model.classifier[0].parameters()},
        {'params': model.classifier[1].parameters()},
        {'params': model.classifier[2].parameters()},
        {'params': model.classifier[3].parameters()},
        {'params': model.classifier[4].parameters()},
        {'params': model.classifier[5].parameters()},
        {'params': model.classifier[6].parameters(),
         'lr': 0.001}], lr=0.0001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)
# Alternative 
#ml = list()
#ml.append({'params': model.modules[14].parameters(), 'lr': 0.1*opt.lr})
#for index, module in enumerate(model.modules):
#   if (index != 14):
#   ml.append({'params': module.parameters()})
#optimizer = optim.Adam(ml, lr=opt.lr, eps=1e-8, weight_decay=1e-5)


In [0]:
class MyModel(nn.Module): 
   def __init__(self, model): 
       super(MyModel, self).__init__() 
       self.model = nn.Sequential(*list(model.features.children())[:-7]) 
       self.fc = torch.nn.Linear(512*4*4, 10) 
   def forward(self, x): 
       x = self.model(x)
       x = x.view(-1, 512 * 4 *4) 
       x = self.fc(x) 
       return x 
 
mymodel = MyModel(model)
# mymodel.modules
mymodel._modules
